In [1]:
# NOTE:
# Renaming variables:
#   - create a dict of ids in a namespace with symtable
#                              (namespaces being: global, class, def)
# with df traversal: rename def locals > rename class locals > rename globals

source = """
import pprint as pp

x = 1

def foo(param):
  print(x)
  x = 2
  param.x = x

class Bar:
  x = 3

"""


In [2]:
import symtable

import pprint as pp

table = symtable.symtable(source, filename='<string>', compile_type='exec')
print(table)
print(table.get_type())
symbols = table.get_symbols()
print(symbols)

children = table.get_children()
print(children)

print(symbols[0].is_referenced())
print(symbols[1].is_local())

fun_children = children[0].get_symbols()
print(fun_children)
print(fun_children[0].is_parameter())
print(fun_children[1].is_local())

<SymbolTable for module <string>>
module
[<symbol 'pp'>, <symbol 'x'>, <symbol 'foo'>, <symbol 'Bar'>]
[<Function SymbolTable for foo in <string>>, <Class SymbolTable for Bar in <string>>]
False
True
[<symbol 'param'>, <symbol 'print'>, <symbol 'x'>]
True
False


In [3]:
import ast
print(ast.dump(ast.parse("my_obj.my_attr")))

Module(body=[Expr(value=Attribute(value=Name(id='my_obj', ctx=Load()), attr='my_attr', ctx=Load()))], type_ignores=[])


In [1]:
# cursed
print("hello")
ppp = print
print = "world"
ppp(print)

hello
world
